# Biodiversity Intactness Index
#### Author: Brooke Grazda
#### [Link to Repository](https://github.com/bgrazda/eds220-final.git)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
from pystac_client import Client  # To access STAC catalogs
import planetary_computer  # To sign items from the MPC STAC catalog 
from IPython.display import Image  # To nicely display images
from geogif import gif
from shapely.geometry import box
import xarray as xr
import os
from pystac import Catalog, get_stac_version


## Import Data

In [13]:
# Import Biodiversity Intactness collection
catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
collection = catalog.get_collection("io-biodiversity")
collection.title

'Biodiversity Intactness'

In [17]:
county_subdiv = gpd.read_file(os.path.join('data', 
                                          'tl_2020_04_cousub.shp'))

In [21]:
bbox = [-112.826843, 32.974108, -111.184387, 33.863574]

In [22]:
arizona = gpd.clip(county_subdiv, bbox)

## Data Exploration

In [24]:
arizona.head()

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
7,04,021,91224,01934941,0402191224,Florence,Florence CCD,22,Z5,G4040,None,None,None,S,2796263519,2035594,+32.7917151,-111.2133547,"POLYGON ((-111.51663 33.11825, -111.50815 33.1..."
9,04,021,93162,01934979,0402193162,San Manuel,San Manuel CCD,22,Z5,G4040,None,None,None,S,3175589809,10542083,+32.8305532,-110.7133450,"POLYGON ((-111.18487 33.08430, -111.18456 33.0..."
29,04,021,90510,01934927,0402190510,Casa Grande,Casa Grande CCD,22,Z5,G4040,None,None,None,S,516681952,303749,+32.8976203,-111.7959903,"POLYGON ((-111.92921 32.98355, -111.92913 32.9..."
32,04,021,90816,01934933,0402190816,Coolidge,Coolidge CCD,22,Z5,G4040,None,None,None,S,308613660,4128677,+32.9295974,-111.5555094,"POLYGON ((-111.61849 32.97411, -111.61849 32.9..."
30,04,021,92091,01934958,0402192091,Maricopa-Stanfield,Maricopa-Stanfield CCD,22,Z5,G4040,None,None,None,S,1026911864,668887,+32.9120190,-112.0749104,"POLYGON ((-112.20300 32.97411, -112.20298 32.9..."


In [25]:
collection

<CollectionClient id=io-biodiversity>

In [27]:
arizona.dtypes

STATEFP       object
COUNTYFP      object
COUSUBFP      object
COUSUBNS      object
GEOID         object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CNECTAFP      object
NECTAFP       object
NCTADVFP      object
FUNCSTAT      object
ALAND          int64
AWATER         int64
INTPTLAT      object
INTPTLON      object
geometry    geometry
dtype: object

In [28]:
arizona.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [30]:
# Get more informaiton about CRS
print('ellipsoid: ', arizona.crs.ellipsoid)
print('datum: ', arizona.crs.datum)
print('is geographic?', arizona.crs.is_geographic)
print('is projected?', arizona.crs.is_projected)

ellipsoid:  GRS 1980
datum:  North American Datum 1983
is geographic? True
is projected? False


### Preliminary Observations

From the initial exploration of both datasets, I learned that the arizona county subdivisions geodataframe has a geographic CRS of GRS 1980. The datum is the North American Datum 9183. I also saw that the majority of the data types are string objects, except for the columns `ALAND` and `AWATER`, which were integers, and the geometries. I viewed the first 5 observations in the arizona geodataframe to ensure that the clipping was correct using the bbox coordinates.

The Biodiversity Intactness Index collection describes its maps to depict biodivrresity intacness over the years 2017 to 2020 with high spatial resolution. The current bounding box for the STAC collection is [-180, -90, 180, 90] so I will have to adjust the bounding box to match that of Arizona county subdivisions.

In [33]:
time_range = '2017-01-01/2020-01-01'

search = catalog.search(
collections = collection,
bbox = bbox,
datetime = time_range)

search

In [34]:
items = search.item_collection()
len(items)

4

In [36]:
item = items[0]

In [38]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [39]:
bii = rioxr.open_rasterio(item.assets['rendered_preview'].href)

/opt/anaconda3/envs/eds220-env/lib/python3.11/site-packages/rioxarray/_io.py:1132: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


In [40]:
bii

<xarray.DataArray (band: 4, y: 1024, x: 1024)> Size: 4MB
[4194304 values with dtype=uint8]
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 8kB 0.5 1.5 2.5 ... 1.022e+03 1.022e+03 1.024e+03
  * y            (y) float64 8kB 0.5 1.5 2.5 ... 1.022e+03 1.022e+03 1.024e+03
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0